In [1]:
import pandas as pd
import os
import numpy as np

from sklearn.feature_selection import RFE

from xgboost import XGBRegressor

os.chdir("D:/WorkPath/PycharmProjects/MutTm-pred")
from Dataset.Process4Dataset.DatasetCeator4PonDT import Dataset4MutTm
from MachineLearning.Util import train_validation_test

最佳预处理参数

In [3]:
dataset = Dataset4MutTm(package_path=r"D:\WorkPath\PycharmProjects\MutTm-pred\Dataset\Process4Dataset",
                          train_dataset_path=r"D:\WorkPath\PycharmProjects\MutTm-pred\Dataset\BasicData\PonDB\pH-Tm\PonDB.csv",
                          test_dataset_path=r"D:\WorkPath\PycharmProjects\MutTm-pred\Dataset\BasicData\ProThermDB\pH-Tm\excllent_ProThermDB_Testing.csv",
                          training_version="PonDB_withpHTm",
                          testing_version="ProThermDBTest_withpHTm",
                          selected_columns=["UniProt_ID", "Mutation", "pH", "Tm", "ΔTm"],
                          features=["neighbor", "aaindex", "group", "param", "hydrop", "context_embedding"],
                          context_length=200,
                          embedding_model_path="DeepLearning/EmbeddingModels/ESM-2/esm2_t33_650M_UR50D",
                          embedding_model_name="ESM-2-650M",
                          embedding_method="add")

train_feature = np.array(dataset.train_feature_set)
train_label = np.array(dataset.train_label_set).ravel()
test_feature = np.array(dataset.test_feature_set)
test_leabel = np.array(dataset.test_label_set).ravel()

当前使用设备：  NVIDIA GeForce RTX 4070 Ti SUPER
===正在从训练集版本为PonDB_withpHTm、测试集版本为ProThermDBTest_withpHTm的原始数据集中进行数据清洗和生物特征提取工作===
1.预处理训练集数据...
-删除数据缺失行及非法行共计0行
-丢弃pH/Tm的缺失值
-前一数据集采用了后一数据集中的0条数据，现已删除
-获取[序列]信息.....该数据集已经经过处理，直接使用缓存文件
-删除了49个UniProtID无法获取序列的数据
-删除了2989条非法长度的数据，当前蛋白质长度被限制在(200, 5000)
-删除条379个突变位点对应错误的数据
2.预处理测试集数据...
-删除数据缺失行及非法行共计0行
-丢弃pH/Tm的缺失值
-获取[序列]信息.....该数据集已经经过处理，直接使用缓存文件
-删除了0个UniProtID无法获取序列的数据
-删除了137条非法长度的数据，当前蛋白质长度被限制在(200, 5000)
-删除条10个突变位点对应错误的数据
3.为训练集数据提取生物特征...
-获取[neighbor特征].....该数据集已经经过处理，直接使用缓存文件
-获取[aaindex特征].....该数据集已经经过处理，直接使用缓存文件
-获取[group特征].....该数据集已经经过处理，直接使用缓存文件


Some weights of EsmModel were not initialized from the model checkpoint at DeepLearning/EmbeddingModels/ESM-2/esm2_t33_650M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-获取[param]特征.....该数据集已经经过处理，直接使用缓存文件
-获取[hydrop]特征.....该数据集已经经过处理，直接使用缓存文件
-获取[context_embedding]特征,该特征没有缓存文件，直接生成...
嵌入模型后显存使用量:  2.494685649871826


KeyboardInterrupt: 

最佳特征筛选

In [ ]:
xgb_rfe_200 = RFE(XGBRegressor(),
                 n_features_to_select=200,
                 step=50,
                 verbose=False)
xgb_rfe_200.fit(train_feature, train_label)

模型调参

In [4]:
from MachineLearning.Util import objective_model
import optuna

study = optuna.create_study(study_name="Best-Model-Parameters-Adjustment-4-28",
                            directions=["minimize", "maximize", "maximize", "minimize"],
                            storage="sqlite:///WinDB/best-4-28.sqlite3",
                            load_if_exists=True)
study.optimize(lambda trial: objective_model(trial, xgb_rfe_200.transform(train_feature), train_label, "XGBoost"), n_trials=300)

[I 2024-04-28 21:52:12,053] A new study created in RDB with name: Best-Model-Parameters-Adjustment-4-28
[I 2024-04-28 21:52:20,192] Trial 0 finished with values: [2.01582412968849, 0.8840053928152954, 0.7764681472588925, 3.6021147931794415] and parameters: {'max_depth': 20, 'min_child_weight': 8, 'gamma': 0.8441098201343432, 'subsample': 0.630632633747725, 'colsample_bytree': 0.738042385779637, 'alpha': 0.7727274861360222, 'lambda': 0.6116793113714573, 'learning_rate': 0.09691809641719272}. 
[I 2024-04-28 21:52:21,885] Trial 1 finished with values: [2.204941235903725, 0.8877675097415828, 0.7785707069752441, 3.5722093964892023] and parameters: {'max_depth': 6, 'min_child_weight': 7, 'gamma': 0.11845670535941166, 'subsample': 0.8613144572232455, 'colsample_bytree': 0.7349417146354769, 'alpha': 0.5830772294697355, 'lambda': 0.4008988292643261, 'learning_rate': 0.05592449542328428}. 
[I 2024-04-28 21:52:28,559] Trial 2 finished with values: [2.238649807215187, 0.8570822135711065, 0.7227096

In [7]:
for trial in study.best_trials:
    print(f"Value: {trial.values}")
    print("Params: ")
    for key, value in trial.params.items():
        print("{}: {}".format(key, value))
    print()

Value: [1.8456685731326439, 0.8947225722833757, 0.7985539950095395, 3.3981409952187116]
Params: 
max_depth: 15
min_child_weight: 7
gamma: 0.5540383649841258
subsample: 0.8348325701085088
colsample_bytree: 0.9276923261227471
alpha: 0.22669961733181054
lambda: 0.08101531588128263
learning_rate: 0.08745884627108118

Value: [1.8703202724274153, 0.8969891576681173, 0.8022212704689983, 3.3642646193592123]
Params: 
max_depth: 9
min_child_weight: 2
gamma: 0.5775160125292507
subsample: 0.8555811516729899
colsample_bytree: 0.964964689271049
alpha: 0.21039593786801636
lambda: 0.6776129379388272
learning_rate: 0.06957736498384784

Value: [1.886369566184896, 0.9002031789756252, 0.8083541349702618, 3.3213214710614545]
Params: 
max_depth: 9
min_child_weight: 6
gamma: 0.25249806752530535
subsample: 0.978228993580981
colsample_bytree: 0.6749603939395634
alpha: 0.5450593595809596
lambda: 0.28113078737150643
learning_rate: 0.09703335755789606

Value: [1.818072889335283, 0.8934361324632979, 0.795652986561

# 保存最好的模型

In [5]:
best_model = XGBRegressor(max_depth=23,
                            min_child_weight=7,
                            gamma=0.67,
                            subsample=0.83,
                            colsample_bytree=0.93,
                            alpha=0.0175)
train_validation_test(best_model,
                       xgb_rfe_200.transform(train_feature),
                       train_label,
                       xgb_rfe_200.transform(test_feature),
                       test_leabel,
                       draw=False)

Starting Training...
Time for Training: 1.4101355075836182
The Result for CV:
MAE:2.0219
PCC:0.8815
R2:0.7748
RMSE:3.5778
Time for CV:5.83445930480957
Starting Testing...
The Result for Testing:MAE:5.4904
PCC:0.5374
R2:0.0105
RMSE:7.2039
Time for Testing:0.0030012130737304688


In [9]:
best_model = XGBRegressor(max_depth=23,
                            min_child_weight=7,
                            gamma=0.67,
                            subsample=0.83,
                            colsample_bytree=0.93,
                            alpha=0.0175)
train_validation_test(best_model,
                       train_feature,
                       train_label,
                       test_feature,
                       test_leabel,
                       draw=False)

Starting Training...
Time for Training: 5.997428894042969
The Result for CV:
MAE:2.0842
PCC:0.8794
R2:0.7727
RMSE:3.6083
Time for CV:28.766226530075073
Starting Testing...
The Result for Testing:MAE:4.0449
PCC:0.7098
R2:0.3809
RMSE:5.6982
Time for Testing:0.003999948501586914


# 保存最佳模型

In [8]:
dataset = Dataset4MutTm(package_path=r"D:\WorkPath\PycharmProjects\MutTm-pred\Dataset\Process4Dataset",
                          train_dataset_path=r"D:\WorkPath\PycharmProjects\MutTm-pred\Dataset\BasicData\PonDB\pH-Tm\PonDB.csv",
                          test_dataset_path=r"D:\WorkPath\PycharmProjects\MutTm-pred\Dataset\BasicData\ProThermDB\pH-Tm\excllent_ProThermDB_Testing.csv",
                          training_version="PonDB_withpHTm",
                          testing_version="ProThermDBTest_withpHTm",
                          selected_columns=["UniProt_ID", "Mutation", "pH", "Tm", "ΔTm"],
                          features=["neighbor", "aaindex", "group", "param", "rpm", "hydrop", 
                                    "context_embedding"],
                          context_length=200,
                          embedding_model_path="DeepLearning/EmbeddingModels/ESM-2/esm2_t33_650M_UR50D",
                          embedding_model_name="ESM-2-650M",
                          embedding_method="add")

train_feature = np.array(dataset.train_feature_set)
train_label = np.array(dataset.train_label_set).ravel()
test_feature = np.array(dataset.test_feature_set)
test_leabel = np.array(dataset.test_label_set).ravel()

当前使用设备：  NVIDIA GeForce RTX 4070 Ti SUPER
===正在从训练集版本为PonDB_withpHTm、测试集版本为ProThermDBTest_withpHTm的原始数据集中进行数据清洗和生物特征提取工作===
1.预处理训练集数据...
-删除数据缺失行及非法行共计0行
-丢弃pH/Tm的缺失值
-前一数据集采用了后一数据集中的0条数据，现已删除
-获取[序列]信息.....该数据集已经经过处理，直接使用缓存文件
-删除了49个UniProtID无法获取序列的数据
-删除了2989条非法长度的数据，当前蛋白质长度被限制在(200, 5000)
-删除条379个突变位点对应错误的数据
2.预处理测试集数据...
-删除数据缺失行及非法行共计0行
-丢弃pH/Tm的缺失值
-获取[序列]信息.....该数据集已经经过处理，直接使用缓存文件
-删除了0个UniProtID无法获取序列的数据
-删除了137条非法长度的数据，当前蛋白质长度被限制在(200, 5000)
-删除条10个突变位点对应错误的数据
3.为训练集数据提取生物特征...
-获取[neighbor特征].....该数据集已经经过处理，直接使用缓存文件
-获取[aaindex特征].....该数据集已经经过处理，直接使用缓存文件
-获取[group特征].....该数据集已经经过处理，直接使用缓存文件
-获取[param]特征.....该数据集已经经过处理，直接使用缓存文件
-获取[pssm]特征.....该数据集已经经过处理，直接使用缓存文件
-获取[hydrop]特征.....该数据集已经经过处理，直接使用缓存文件


Some weights of EsmModel were not initialized from the model checkpoint at DeepLearning/EmbeddingModels/ESM-2/esm2_t33_650M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-获取[context_embedding]特征,该特征没有缓存文件，直接生成...
嵌入模型后显存使用量:  5.004342079162598
-释放模型后显存用量: 2.5128350257873535
4.为测试集数据提取生物特征...
-获取[neighbor特征].....该数据集已经经过处理，直接使用缓存文件
-获取[aaindex特征].....该数据集已经经过处理，直接使用缓存文件
-获取[group特征].....该数据集已经经过处理，直接使用缓存文件
-获取[param]特征.....该数据集已经经过处理，直接使用缓存文件
-获取[pssm]特征.....该数据集已经经过处理，直接使用缓存文件
-获取[hydrop]特征.....该数据集已经经过处理，直接使用缓存文件
-获取[context_embedding]特征,该特征没有缓存文件，直接生成...


Some weights of EsmModel were not initialized from the model checkpoint at DeepLearning/EmbeddingModels/ESM-2/esm2_t33_650M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


嵌入模型后显存使用量:  5.004342079162598
-释放模型后显存用量: 2.5128350257873535
6.从全数据集中提取生物特征集、标签集和基本信息集...
7.数据清洗和生物特征提取工作完成==>当前显存用量:2.5096564292907715


In [11]:
best_model = XGBRegressor(max_depth=23,
                        min_child_weight=7,
                        gamma=0.67,
                        subsample=0.83,
                        colsample_bytree=0.93,
                        alpha=0.0175)
best_model.fit(train_feature, train_label)
best_model.save_model("BestModel.json")

In [12]:
# load the model
load_model = XGBRegressor()
load_model.load_model("BestModel.json")

In [13]:
from MachineLearning.Util import test
test(load_model,
     test_feature,
     test_leabel,
     draw=False)

Starting Testing...
The Result for Testing:MAE:4.0449
PCC:0.7098
R2:0.3809
RMSE:5.6982
Time for Testing:0.003999948501586914


In [15]:
pred_label = load_model.predict(test_feature)
pd.concat([pd.Series(test_leabel), pd.Series(pred_label)], axis=1)

,0,1
0,7.50,8.161337
1,-1.00,3.761423
2,-6.90,-5.470471
3,-4.58,-3.524244
4,-2.80,-4.950304
...,...,...
208,-10.50,-8.000441
209,-9.00,-11.107576
210,-15.30,-8.471524
211,-12.80,-6.315666


In [20]:
importance_df = pd.DataFrame({'Feature': dataset.train_feature_set.columns, 'Importance': load_model.feature_importances_})
importance_df

,Feature,Importance
0,pH,0.000298
1,Tm,0.000858
2,neighbor1,0.000097
3,neighbor2,0.000631
4,neighbor3,0.000178
...,...,...
1340,embedding198,0.000347
1341,embedding199,0.000051
1342,embedding200,0.001518
1343,PA,0.001477
